# Side-by-Side Code Comparison - Part 1D

This notebook provides a detailed side-by-side comparison between the original 2024 implementation and our modernized 2025 version. This is perfect for understanding exactly what changed and why.

## 📋 Project Structure Comparison

### Original 2024 Structure:
```
v1-162-part1/
├── app/
│   ├── server.py          # LangServe-based
│   └── rag_chain.py       # Complex with TypedDict
├── rag-data-loader/
│   └── rag_load_and_process.py  # Required flattening
├── pdf-documents/
├── pyproject.toml         # Python 3.11, LangServe dependency
└── .env
```

### Modern 2025 Structure:
```
v2-modern-step1/
├── app/
│   ├── server.py          # Direct FastAPI
│   └── rag_chain.py       # Clean, simple
├── rag-data-loader/
│   └── rag_load_and_process.py  # Direct processing
├── pdf-documents/
├── pyproject.toml         # Python 3.13, modern deps
├── .env.template          # Template for easy setup
└── .gitignore            # Better exclusions
```

## 1. Dependencies Comparison

### 🗓️ Original 2024 `pyproject.toml`:

In [ ]:
original_2024_deps = """
[tool.poetry]
name = "164-rag-for-pdfs-v2"
version = "0.1.0"

[tool.poetry.dependencies]
python = ">=3.11,<3.12"           # Constrained by unstructured
uvicorn = "^0.23.2"               # Older version
langserve = {extras = ["server"], version = ">=0.0.30"}  # DEPRECATED!
pydantic = "<2"                   # LangServe constraint
tqdm = "^4.66.2"
unstructured = {extras = ["all-docs"], version = "^0.12.6"}
langchain-experimental = "^0.0.55"
python-dotenv = "^1.0.1"
langchain-openai = "^0.1.1"       # Older version
langchain-community = "^0.0.31"   # Older version
tiktoken = "^0.6.0"
psycopg = "^3.1.18"
pgvector = "^0.2.5"
"""

print("📜 Original 2024 Dependencies:")
print(original_2024_deps)

In [ ]:
modern_2025_deps = """
[tool.poetry]
name = "v2-modern-rag-step1"
version = "0.1.0"
description = "Modern RAG application for PDFs - Step 1 (2025 update)"

[tool.poetry.dependencies]
python = ">=3.13,<3.14"           # Latest Python!
fastapi = "^0.115.0"              # Direct FastAPI (no LangServe)
uvicorn = "^0.32.0"               # Latest version
python-dotenv = "^1.0.1"
langchain-core = "^0.3.0"         # Modular approach
langchain-openai = "^0.2.0"       # Latest version
langchain-community = "^0.3.0"    # Latest version
langchain-experimental = "^0.3.0" # Latest version
tqdm = "^4.66.0"
unstructured = {extras = ["all-docs"], version = "^0.16.0"}
tiktoken = "^0.8.0"               # Latest version
psycopg = "^3.2.0"               # Latest version
pgvector = "^0.3.0"              # Latest version
# NO pydantic constraints!
# NO langserve dependency!
"""

print("🆕 Modern 2025 Dependencies:")
print(modern_2025_deps)

## 2. Data Processing Comparison

### 🗓️ Original 2024 `rag_load_and_process.py`:

In [ ]:
original_2024_processing = '''
import os
from dotenv import load_dotenv
from langchain_community.document_loaders import DirectoryLoader, UnstructuredPDFLoader
from langchain_community.vectorstores.pgvector import PGVector
from langchain_experimental.text_splitter import SemanticChunker
from langchain_openai import OpenAIEmbeddings

load_dotenv()

loader = DirectoryLoader(
    os.path.abspath("../pdf-documents"),
    glob="**/*.pdf",
    use_multithreading=True,
    show_progress=True,
    max_concurrency=50,
    loader_cls=UnstructuredPDFLoader,
)
docs = loader.load()

# OLD EXPENSIVE MODEL!
embeddings = OpenAIEmbeddings(model='text-embedding-ada-002')

text_splitter = SemanticChunker(embeddings=embeddings)

# WEIRD FLATTENING STEP REQUIRED!
flattened_docs = [doc[0] for doc in docs if doc]
chunks = text_splitter.split_documents(flattened_docs)

PGVector.from_documents(
    documents=chunks,
    embedding=embeddings,
    collection_name="collection164",
    connection_string="postgresql+psycopg://postgres@localhost:5432/database164",
    pre_delete_collection=True,
)
'''

print("📜 Original 2024 Data Processing:")
print(original_2024_processing)

In [ ]:
modern_2025_processing = '''
import os
from dotenv import load_dotenv
from langchain_community.document_loaders import DirectoryLoader, UnstructuredPDFLoader
from langchain_community.vectorstores.pgvector import PGVector
from langchain_experimental.text_splitter import SemanticChunker
from langchain_openai import OpenAIEmbeddings

load_dotenv()

loader = DirectoryLoader(
    os.path.abspath("../pdf-documents"),
    glob="**/*.pdf",
    use_multithreading=True,
    show_progress=True,
    max_concurrency=50,
    loader_cls=UnstructuredPDFLoader,
)
docs = loader.load()

# MODERN CHEAP MODEL! (5x cheaper)
embeddings = OpenAIEmbeddings(model='text-embedding-3-small')

text_splitter = SemanticChunker(embeddings=embeddings)

# NO FLATTENING NEEDED! Direct processing
chunks = text_splitter.split_documents(docs)

print(f"Created {len(chunks)} chunks from {len(docs)} documents")

PGVector.from_documents(
    documents=chunks,
    embedding=embeddings,
    collection_name="modern_rag_collection",
    connection_string="postgresql+psycopg://postgres@localhost:5432/modern_rag_db",
    pre_delete_collection=True,
)

print("Vector database created successfully!")
'''

print("🆕 Modern 2025 Data Processing:")
print(modern_2025_processing)

## 3. RAG Chain Comparison

### 🗓️ Original 2024 `rag_chain.py`:

In [ ]:
original_2024_chain = '''
import os
from operator import itemgetter
from typing import TypedDict

from dotenv import load_dotenv
from langchain_community.vectorstores.pgvector import PGVector
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI, OpenAIEmbeddings

load_dotenv()

vector_store = PGVector(
    collection_name="collection164",
    connection_string="postgresql+psycopg://postgres@localhost:5432/database164",
    embedding_function=OpenAIEmbeddings()  # Default ada-002
)

template = """
Answer given the following context:
{context}

Question: {question}
"""

ANSWER_PROMPT = ChatPromptTemplate.from_template(template)

# EXPENSIVE MODEL!
llm = ChatOpenAI(temperature=0, model='gpt-4-1106-preview', streaming=True)

# COMPLEX LANGSERVE-SPECIFIC SETUP!
class RagInput(TypedDict):
    question: str

final_chain = (
    {
    "context": (itemgetter("question") | vector_store.as_retriever()),
    "question": itemgetter("question")
    }
    | ANSWER_PROMPT
    | llm
    | StrOutputParser()
).with_types(input_type=RagInput)  # LangServe requirement!
'''

print("📜 Original 2024 RAG Chain:")
print(original_2024_chain)

In [ ]:
modern_2025_chain = '''
import os
from dotenv import load_dotenv
from langchain_community.vectorstores.pgvector import PGVector
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import ChatOpenAI, OpenAIEmbeddings

load_dotenv()

# MODERN EMBEDDINGS!
embeddings = OpenAIEmbeddings(model='text-embedding-3-small')

vector_store = PGVector(
    collection_name="modern_rag_collection",
    connection_string="postgresql+psycopg://postgres@localhost:5432/modern_rag_db",
    embedding_function=embeddings
)

retriever = vector_store.as_retriever(search_kwargs={"k": 4})

template = """Answer the question based on the following context:

Context: {context}

Question: {question}

Answer: """

prompt = ChatPromptTemplate.from_template(template)

# MUCH CHEAPER MODEL!
llm = ChatOpenAI(
    temperature=0, 
    model='gpt-4o-mini',  # 67x cheaper!
    streaming=True
)

# CLEAN, SIMPLE APPROACH!
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

rag_chain = (
    {
        "context": retriever | format_docs,
        "question": RunnablePassthrough()
    }
    | prompt
    | llm
    | StrOutputParser()
)
# No TypedDict needed!
# No itemgetter complexity!
# No LangServe requirements!
'''

print("🆕 Modern 2025 RAG Chain:")
print(modern_2025_chain)

## 4. Server Implementation Comparison

### 🗓️ Original 2024 `server.py`:

In [ ]:
original_2024_server = '''
from fastapi import FastAPI
from fastapi.responses import RedirectResponse
from langserve import add_routes  # DEPRECATED!
from app.rag_chain import final_chain

app = FastAPI()

@app.get("/")
async def redirect_root_to_docs():
    return RedirectResponse("/docs")

# BLACK BOX - NO CONTROL!
add_routes(app, final_chain, path="/rag")

if __name__ == "__main__":
    import uvicorn
    uvicorn.run(app, host="0.0.0.0", port=8000)
    
# Problems with this approach:
# ❌ No custom error handling
# ❌ No request/response validation
# ❌ No streaming control
# ❌ Deprecated LangServe dependency
# ❌ Limited customization
'''

print("📜 Original 2024 Server:")
print(original_2024_server)

In [ ]:
modern_2025_server = '''
from fastapi import FastAPI, HTTPException
from fastapi.responses import RedirectResponse
from pydantic import BaseModel
from typing import AsyncGenerator
import asyncio
import json

from app.rag_chain import rag_chain

app = FastAPI(
    title="Modern RAG API",
    description="A modern RAG application for querying PDF documents (2025 update)",
    version="2.0.0"
)

# PROPER REQUEST/RESPONSE MODELS
class QueryRequest(BaseModel):
    question: str

class QueryResponse(BaseModel):
    answer: str

@app.get("/")
async def redirect_root_to_docs():
    return RedirectResponse("/docs")

# FULL CONTROL WITH PROPER ERROR HANDLING
@app.post("/query", response_model=QueryResponse)
async def query_documents(request: QueryRequest):
    try:
        answer = await rag_chain.ainvoke(request.question)
        return QueryResponse(answer=answer)
    except Exception as e:
        raise HTTPException(status_code=500, detail=f"Error: {str(e)}")

# STREAMING SUPPORT!
@app.post("/stream")
async def stream_query(request: QueryRequest):
    async def generate_response():
        try:
            async for chunk in rag_chain.astream(request.question):
                yield f"data: {json.dumps({'chunk': chunk})}\n\n"
        except Exception as e:
            yield f"data: {json.dumps({'error': str(e)})}\n\n"
    
    from fastapi.responses import StreamingResponse
    return StreamingResponse(generate_response(), media_type="text/plain")

# HEALTH CHECK FOR MONITORING
@app.get("/health")
async def health_check():
    return {"status": "healthy", "version": "2.0.0"}

if __name__ == "__main__":
    import uvicorn
    uvicorn.run(app, host="0.0.0.0", port=8000)
    
# Benefits of this approach:
# ✅ Custom error handling
# ✅ Proper request/response validation
# ✅ Streaming support
# ✅ No deprecated dependencies
# ✅ Full customization control
# ✅ Health check endpoint
# ✅ Better API documentation
'''

print("🆕 Modern 2025 Server:")
print(modern_2025_server)

## 5. Cost Analysis Comparison

Let's calculate the real-world cost differences:

In [ ]:
# Cost comparison calculator
print("💰 COST ANALYSIS: 2024 vs 2025\n")

# Assumptions for a typical RAG query
docs_to_embed = 100  # Number of document chunks
tokens_per_chunk = 500  # Average tokens per chunk
queries_per_day = 100  # Daily queries
context_tokens = 2000  # Tokens in context per query
response_tokens = 200  # Average response tokens

# 2024 Pricing
ada_002_price = 0.0001  # per 1k tokens
gpt4_preview_input = 0.01  # per 1k tokens
gpt4_preview_output = 0.03  # per 1k tokens

# 2025 Pricing  
embedding_3_small_price = 0.00002  # per 1k tokens
gpt4o_mini_input = 0.00015  # per 1k tokens
gpt4o_mini_output = 0.0006  # per 1k tokens

# Calculate embedding costs (one-time setup)
total_embedding_tokens = docs_to_embed * tokens_per_chunk
embedding_2024 = total_embedding_tokens * ada_002_price / 1000
embedding_2025 = total_embedding_tokens * embedding_3_small_price / 1000

print(f"📊 ONE-TIME EMBEDDING COSTS ({docs_to_embed} chunks, {tokens_per_chunk} tokens each):")
print(f"  2024 (ada-002): ${embedding_2024:.4f}")
print(f"  2025 (3-small): ${embedding_2025:.4f}")
print(f"  💡 Embedding savings: {((embedding_2024-embedding_2025)/embedding_2024)*100:.1f}%\n")

# Calculate daily LLM costs
daily_input_tokens = queries_per_day * context_tokens
daily_output_tokens = queries_per_day * response_tokens

llm_2024_daily = (daily_input_tokens * gpt4_preview_input / 1000) + (daily_output_tokens * gpt4_preview_output / 1000)
llm_2025_daily = (daily_input_tokens * gpt4o_mini_input / 1000) + (daily_output_tokens * gpt4o_mini_output / 1000)

print(f"📈 DAILY LLM COSTS ({queries_per_day} queries/day):")
print(f"  2024 (gpt-4-preview): ${llm_2024_daily:.4f}/day")
print(f"  2025 (gpt-4o-mini): ${llm_2025_daily:.4f}/day")
print(f"  💡 Daily LLM savings: {((llm_2024_daily-llm_2025_daily)/llm_2024_daily)*100:.1f}%\n")

# Monthly and yearly projections
monthly_2024 = llm_2024_daily * 30
monthly_2025 = llm_2025_daily * 30
yearly_2024 = llm_2024_daily * 365
yearly_2025 = llm_2025_daily * 365

print(f"📅 MONTHLY COSTS:")
print(f"  2024: ${monthly_2024:.2f}/month")
print(f"  2025: ${monthly_2025:.2f}/month")
print(f"  💰 Monthly savings: ${monthly_2024-monthly_2025:.2f}")

print(f"\n🗓️ YEARLY COSTS:")
print(f"  2024: ${yearly_2024:.2f}/year")
print(f"  2025: ${yearly_2025:.2f}/year")
print(f"  💰 Yearly savings: ${yearly_2024-yearly_2025:.2f}")

total_savings_percent = ((yearly_2024 - yearly_2025) / yearly_2024) * 100
print(f"\n🎯 TOTAL COST REDUCTION: {total_savings_percent:.1f}%")

## 6. Feature Comparison Matrix

### Complete Feature Analysis:

In [ ]:
import pandas as pd

# Create comprehensive comparison
comparison_data = {
    'Feature': [
        'Python Version',
        'Dependency Management',
        'Embedding Model',
        'LLM Model',
        'Deployment Framework',
        'Error Handling',
        'Request Validation',
        'Streaming Support',
        'API Documentation',
        'Health Checks',
        'Code Complexity',
        'Maintainability',
        'Future-Proof',
        'Cost Efficiency',
        'Performance'
    ],
    '2024 Approach': [
        'Python 3.11.4',
        'Poetry with constraints',
        'text-embedding-ada-002',
        'gpt-4-1106-preview',
        'LangServe (deprecated)',
        'Limited/default',
        'Basic TypedDict',
        'Basic via LangServe',
        'Auto-generated',
        'None',
        'High (TypedDict, itemgetter)',
        'Low (complex abstractions)',
        'Poor (deprecated deps)',
        'Expensive',
        'Good'
    ],
    '2025 Approach': [
        'Python 3.13.3',
        'Clean Poetry setup',
        'text-embedding-3-small',
        'gpt-4o-mini',
        'Direct FastAPI',
        'Custom with details',
        'Full Pydantic models',
        'Custom streaming',
        'Rich, customizable',
        'Built-in endpoints',
        'Low (clean patterns)',
        'High (readable code)',
        'Excellent (modern stack)',
        'Very cost-effective',
        'Excellent'
    ],
    'Improvement': [
        '⚡ Latest features & performance',
        '🧹 Fewer dependency conflicts',
        '💰 5x cost reduction',
        '💰 67x cost reduction',
        '🎯 Full control, no deprecation',
        '🐛 Better debugging experience',
        '✅ Type safety & validation',
        '🌊 Real-time user experience',
        '📖 Better developer experience',
        '📊 Production monitoring',
        '🧹 Easier to understand',
        '🔧 Easier to modify',
        '🚀 Long-term viability',
        '💸 Significant cost savings',
        '⚡ Faster responses'
    ]
}

df = pd.DataFrame(comparison_data)
print("📊 COMPREHENSIVE FEATURE COMPARISON:\n")
print(df.to_string(index=False))

## 7. Migration Guide

### For Students Working with the Old Version:

If you have the 2024 version and want to migrate:

In [ ]:
migration_steps = """
🔄 MIGRATION STEPS FROM 2024 TO 2025:

1. 🐍 UPDATE PYTHON:
   - Install Python 3.13.3
   - Update Poetry to 2.1.4

2. 📦 UPDATE DEPENDENCIES:
   - Remove: langserve, pydantic constraints
   - Add: fastapi (direct)
   - Update: all langchain-* packages to latest

3. 🔧 UPDATE CODE:
   - Replace LangServe with FastAPI endpoints
   - Change embedding model to text-embedding-3-small
   - Change LLM to gpt-4o-mini
   - Remove document flattening step
   - Simplify chain structure

4. 🗄️ UPDATE DATABASE:
   - Create new database (or clean existing)
   - Re-run data loading with new models

5. ✅ TEST:
   - Verify all functionality works
   - Test API endpoints
   - Monitor costs and performance

⏱️ ESTIMATED TIME: 2-4 hours
💰 COST REDUCTION: ~95% for typical usage
"""

print(migration_steps)

## 8. Learning Outcomes

### What You Should Take Away:

In [ ]:
learning_outcomes = """
🎓 KEY LEARNING OUTCOMES:

1. 🚀 AI MOVES FAST:
   - Technologies can become deprecated in months
   - Always check for newer, better alternatives
   - Stay updated with the AI community

2. 💰 COST OPTIMIZATION MATTERS:
   - Model choices dramatically affect costs
   - Newer models often provide better value
   - Always calculate total cost of ownership

3. 🧹 SIMPLICITY WINS:
   - Direct approaches often beat complex abstractions
   - Fewer dependencies = fewer problems
   - Readable code is maintainable code

4. 🎯 CONTROL IS VALUABLE:
   - Black box solutions limit customization
   - Direct implementations offer more flexibility
   - Custom error handling improves debugging

5. 🔮 FUTURE-PROOFING:
   - Choose actively maintained technologies
   - Avoid deprecated dependencies
   - Design for change

6. 📊 MONITORING MATTERS:
   - Production systems need health checks
   - Track performance and costs
   - Plan for scale

🎯 BOTTOM LINE:
The 2025 approach is not just "newer" - it's fundamentally
better in terms of cost, performance, maintainability, and
future-proofing. This is how modern AI applications should
be built.
"""

print(learning_outcomes)

## Summary

🎉 **You've now seen a complete side-by-side comparison!**

### The Modernization Journey:
- 📚 **Started**: Understanding the original 2024 approach
- 🔍 **Analyzed**: What changed and why in the AI ecosystem
- 🛠️ **Implemented**: A modern, cost-effective solution
- 📊 **Compared**: Old vs new approaches in detail

### Key Improvements Achieved:
- 💰 **~95% cost reduction** through model optimization
- 🧹 **Simplified codebase** with modern patterns
- 🎯 **Better control** with direct FastAPI implementation
- 🚀 **Future-proof** architecture using current best practices

### Next Steps:
1. **Experiment** with the modern implementation
2. **Try different models** and compare results
3. **Apply these principles** to your own AI projects
4. **Stay updated** with the rapidly evolving AI landscape

The AI field moves incredibly fast, but by understanding the principles behind these modernizations, you'll be better equipped to adapt to future changes. Happy building! 🚀